# ETS Example with Numpyro

This is a working example to compare results on ETS model between estimator `Numpyro` and estimator `PyStan`.

In [1]:
import pandas as pd
import numpy as np
import orbit
import matplotlib.pyplot as plt

from orbit.utils.dataset import load_iclaims
from orbit.diagnostics.plot import plot_predicted_data, plot_predicted_components, params_comparison_boxplot
from orbit.utils.plot import get_orbit_style
plt.style.use(get_orbit_style())
from orbit.models import ETS

In [2]:
orbit.__version__

'1.1.3dev'

In [3]:
print(np.__version__)

1.22.2


In [4]:
%load_ext autoreload
%autoreload 2

## Data

In [5]:
# can also consider transform=False
raw_df = load_iclaims(transform=True)
raw_df.dtypes

week              datetime64[ns]
claims                   float64
trend.unemploy           float64
trend.filling            float64
trend.job                float64
sp500                    float64
vix                      float64
dtype: object

In [6]:
df = raw_df.copy()
df.head()

,week,claims,trend.unemploy,trend.filling,trend.job,sp500,vix
0,2010-01-03,13.386595,0.219882,-0.318452,0.117500,-0.417633,0.122654
1,2010-01-10,13.624218,0.219882,-0.194838,0.168794,-0.425480,0.110445
2,2010-01-17,13.398741,0.236143,-0.292477,0.117500,-0.465229,0.532339
3,2010-01-24,13.137549,0.203353,-0.194838,0.106918,-0.481751,0.428645
4,2010-01-31,13.196760,0.134360,-0.242466,0.074483,-0.488929,0.487404


In [7]:
test_size=52
train_df=df[:-test_size]
test_df=df[-test_size:]

## Comparing MCMC Results

If users see a warning messages about `tree_flatten(xs)`, feel free to ignore them.

In [8]:
ets = ETS(response_col='claims',
          date_col='week',
          seasonality=52,
          seed=2020,
          estimator='numpyro-mcmc',
          num_warmup=3000,
          num_sample=1000,
          chains=4,
)
ets.fit(train_df);

IllegalArgument: Invalid estimator. Must be one of ['stan-map', 'stan-mcmc']

In [ ]:
predicted_df = ets.predict(df=df, decompose=True)
predicted_df

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df,
                        predicted_df=predicted_df,
                        date_col='week',
                        actual_col='claims',
                        test_actual_df=test_df)

In [ ]:
_ = plot_predicted_components(predicted_df=predicted_df, date_col='week')

In [ ]:
numpyro_ps = ets.get_posterior_samples()

In [ ]:
ets = ETS(response_col='claims',
          date_col='week',
          seasonality=52,
          seed=2020,
          estimator='stan-mcmc',
          num_warmup=3000,
          num_sample=1000,
          chains=4,
)
ets.fit(train_df);

In [ ]:
predicted_df = ets.predict(df=df, decompose=True)
predicted_df

In [ ]:
stan_ps = ets.get_posterior_samples()

In [ ]:
params_comparison_boxplot(
        [numpyro_ps, stan_ps],
        var_names=['lev_sm', 'obs_sigma', 'sea_sm'],
        model_names=["numpyro", "stan"],
        box_width = .1, box_distance=0.1,
        showfliers=True
);

## Comparing MAP Results

In [ ]:
ets = ETS(response_col='claims',
          date_col='week',
          seasonality=52,
          seed=2020,
          estimator='numpyro-map',
          num_iters=30000,
)
ets.fit(train_df);
numpyro_posteriors = ets.get_point_posteriors()['map']

In [ ]:
ets = ETS(response_col='claims',
          date_col='week',
          seasonality=52,
          seed=2020,
          estimator='stan-map',
          num_iters=5000,
)
ets.fit(train_df);
stan_posteriors = ets.get_point_posteriors()['map']

In [ ]:
print("----NumPyro---")
for param in ['lev_sm', 'obs_sigma', 'sea_sm']:
    print('{}: {:.5f}'.format(param, numpyro_posteriors[param].mean()))
print("----PyStan---")  
for param in ['lev_sm', 'obs_sigma', 'sea_sm']:
    print('{}: {:.5f}'.format(param, stan_posteriors[param].mean()))